### Install Necessary Libraries
This cell installs the `openai` library (version 0.28) which is used to interact with OpenAI's API. `requests` and `python-dotenv` are also listed, but usually pre-installed or handled by Colab.

In [1]:

pip install openai==0.28 requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.14.0
    Uninstalling openai-2.14.0:
      Successfully uninstalled openai-2.14.0


### Weather API Integration (`get_weather` function)
This cell defines the `get_weather` function and demonstrates its usage.
- It uses the `requests` library to make a call to the WeatherAPI.com service.
- It retrieves the API key for WeatherAPI from `google.colab.userdata` (named 'WEATHER_API_KEY').
- The function constructs a URL with the provided location and API key.
- `response.raise_for_status()` checks if the request was successful; if not, it raises an exception.
- It returns the weather data in JSON format.
- The `if __name__ == "__main__"` block serves as a test to fetch and print weather information for 'London'.

In [14]:
import requests
from google.colab import userdata
y = userdata.get('WEATHER_API_KEY')

def get_weather(location):
    try:
        url = f"http://api.weatherapi.com/v1/current.json"
        params = {
            "key": y,
            "q": location,
            "aqi": "no"
        }
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"Weather API error: {str(e)}")

# Test weather API integration
if __name__ == "__main__":
    try:
        weather_data = get_weather("London")
        print(f"Weather in {weather_data['location']['name']}:")
        print(f"Temperature: {weather_data['current']['temp_c']}°C")
        print(f"Condition: {weather_data['current']['condition']['text']}")
    except Exception as e:
        print(f"Error: {e}")

Weather in London:
Temperature: 7.2°C
Condition: Overcast


### OpenAI Integration and Weather Agent (`weather_agent` function)
This is the core cell that integrates with OpenAI's function calling feature and orchestrates the weather information retrieval.

#### API Key Setup
- It imports `openai` and `google.colab.userdata`.
- It retrieves the OpenAI API key from `google.colab.userdata` (named 'OPENAI_NEW') and sets `openai.api_key`.

#### Function Definition for OpenAI
- The `functions` list defines a schema for the `get_weather` function that OpenAI can understand.
- This schema tells OpenAI that there's a function named `get_weather` that takes a `location` (string) as a required parameter.

#### `get_weather_emoji` function
- This helper function takes a weather condition text (e.g., "Overcast", "Clear") and returns a corresponding emoji (e.g., ☁️, ☀️) for visual representation.

#### `weather_agent` function
- This is the main function that takes a `user_query` (e.g., "What's the weather like in New York?").
- It uses `openai.ChatCompletion.create` to send the user's query to the GPT-4 model, along with the defined `functions`.
- OpenAI's `function_call="auto"` capability allows the model to decide whether to call one of the provided functions.
- If OpenAI decides to call `get_weather`, it extracts the `function_call.arguments` (which is a JSON string) and parses it to get the `location`.
- It then calls the local `get_weather` function (defined in the previous cell) with the extracted location.
- **Error Handling**: A `try-except` block specifically catches a "400 Client Error" from the weather API, indicating an invalid city name, and returns a user-friendly error message.
- It uses the `get_weather_emoji` function to get an emoji for the current weather condition.
- Finally, it formats and returns a detailed, multi-line weather report including the emoji, location, last updated time, temperature (with 'feels like'), condition, wind speed/direction, and humidity.

#### Testing the `weather_agent`
- The `if __name__ == "__main__"` block includes a series of `test_queries` to demonstrate the `weather_agent`'s functionality, including successful calls and an invalid city query to test error handling.

In [18]:
import openai
from google.colab import userdata
import json # Keep json import for parsing function arguments

x = userdata.get('OPENAI_NEW')
openai.api_key = x

# Define the function that OpenAI can call
functions = [
    {
        "name": "get_weather",
        "description": "Get current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City name or location"
                }
            },
            "required": ["location"]
        }
    }
]

# Test OpenAI function definition
#if __name__ == "__main__":
 #   try:
 #       response = openai.ChatCompletion.create(
 #           model="gpt-4",
 #           messages=[{"role": "user", "content": "What's the weather like in Tokyo?"}],
 #           functions=functions,
 #           function_call="auto"
 #       )
 #       print("OpenAI Response:")
 #       print(response.choices[0].message.function_call)
 #   except Exception as e:
 #       print(f"Error: {e}")

def get_weather_emoji(condition_text):
    condition_text_lower = condition_text.lower()
    if "clear" in condition_text_lower or "sunny" in condition_text_lower:
        return "☀️"
    elif "cloudy" in condition_text_lower or "overcast" in condition_text_lower:
        return "☁️"
    elif "rain" in condition_text_lower or "drizzle" in condition_text_lower:
        return "🌧️"
    elif "snow" in condition_text_lower or "sleet" in condition_text_lower:
        return "🌨️"
    elif "thunder" in condition_text_lower:
        return "⛈️"
    elif "fog" in condition_text_lower or "mist" in condition_text_lower:
        return "🌫️"
    else:
        return "🌡️" # Default emoji


def weather_agent(user_query):
    try:
        # Get function call from OpenAI
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": user_query}],
            functions=functions,
            function_call="auto"
        )

        # Extract function call
        function_call = response.choices[0].message.function_call

        # Execute function
        if function_call.name == "get_weather":
            # Parse the arguments from JSON string
            arguments = json.loads(function_call.arguments)
            location = arguments["location"]
            try:
                weather_data = get_weather(location)
            except Exception as e:
                if "400 Client Error" in str(e):
                    return f"Error: Could not find weather for '{location}'. Please check the city name."
                else:
                    raise

            # Get emoji for current condition
            weather_emoji = get_weather_emoji(weather_data['current']['condition']['text'])

            # Format response with additional details including emoji and last updated time
            return (
                f"{weather_emoji} Current weather in {weather_data['location']['name']} at {weather_data['current']['last_updated']}:\n"
                f"  Temperature: {weather_data['current']['temp_c']}°C (feels like {weather_data['current']['feelslike_c']}°C)\n"
                f"  Condition: {weather_data['current']['condition']['text']}\n"
                f"  Wind: {weather_data['current']['wind_kph']} kph {weather_data['current']['wind_dir']}\n"
                f"  Humidity: {weather_data['current']['humidity']}%" )
    except Exception as e:
        return f"Error: {str(e)}"

    # Test the complete weather agent
if __name__ == "__main__":
    test_queries = [
        "What's the weather like in Singapore?",
        "Is it raining in London right now?",
        "Tell me the temperature in New York",
        "How hot is the weather tonight in Sydney?",
        "What's the weather like in a nonexistantcity?"
    ]

    for query in test_queries:
        print(f"\nQuery: {query}")
        print(f"Response: {weather_agent(query)}")


Query: What's the weather like in Singapore?
Response: ☁️ Current weather in Singapore at 2026-01-16 18:00:
  Temperature: 30.1°C (feels like 36.1°C)
  Condition: Overcast
  Wind: 14.8 kph NNE
  Humidity: 62%

Query: Is it raining in London right now?
Response: ☁️ Current weather in London at 2026-01-16 10:00:
  Temperature: 7.2°C (feels like 4.5°C)
  Condition: Overcast
  Wind: 14.8 kph S
  Humidity: 93%

Query: Tell me the temperature in New York
Response: ☀️ Current weather in New York at 2026-01-16 05:00:
  Temperature: -5.6°C (feels like -13.7°C)
  Condition: Clear
  Wind: 29.2 kph WNW
  Humidity: 43%

Query: How hot is the weather tonight in Sydney?
Response: ☁️ Current weather in Sydney at 2026-01-16 21:00:
  Temperature: 22.3°C (feels like 24.7°C)
  Condition: Partly cloudy
  Wind: 24.1 kph SE
  Humidity: 83%

Query: What's the weather like in a nonexistantcity?
Response: Error: Could not find weather for 'nonexistantcity'. Please check the city name.


In [ ]:
# Extension activities